In [1]:
import pandas as pd
from glob import glob
import os
import numpy as np
import json
import sys
from tqdm import tqdm

In [2]:
def get_labels(aphasia_type):
   """
   aphasia_type에 따른 여러 레이블을 반환하는 함수
   
   Parameters:
       aphasia_type (str): WAB 기반 실어증 타입
       
   Returns:
       type (str): 표준화된 실어증 타입명
       status (str): Control 또는 Aphasia 상태
       ct_label (int): Control(0) vs Treatment(1) 레이블
       wab_label (int): Western Aphasia Battery 기반 레이블
       type_label (int): Control(0), Fluent(1), Non-Comprehension(2), Non-Fluent(3)
       flu_label (int): Non-Fluent(0) vs Fluent(1)  
       com_label (int): Comprehends(0) vs Not Comprehends(1)
   """
   
   # 소문자로 변환하여 처리
   aphasia_type = aphasia_type.lower()
   
   # 기본값 설정
   type = ""
   ct_label = 0
   wab_label = 0 
   type_label = 0
   flu_label = 0
   com_label = 0
   
   if aphasia_type in ['control', 'notaphasicbywab']:
       type = "Control"
       status = 'Control'
       ct_label = 0
       wab_label = 0
       type_label = 0
       flu_label = 0
       com_label = 0
       
   elif aphasia_type in ['anomic', 'conduction']:
       type = aphasia_type.capitalize()
       status = 'Aphasia'
       ct_label = 1
       wab_label = 1  # Fluent & Comprehends
       type_label = 1  # Fluent
       flu_label = 1  # Fluent
       com_label = 0  # Comprehends
       
   elif aphasia_type in ['wernicke', 'transsensory']:
       type = "Wernicke" if aphasia_type == 'wernicke' else "Trans. Sensory"
       status = 'Aphasia'
       ct_label = 1
       wab_label = 2  # Fluent & Not Comprehends
       type_label = 2  # Non-Comprehension
       flu_label = 1  # Fluent
       com_label = 1  # Not Comprehends
       
   elif aphasia_type in ['broca', 'transmotor']:
       type = "Broca" if aphasia_type == 'broca' else "Trans. Motor"
       status = 'Aphasia'
       ct_label = 1
       wab_label = 3  # Non-Fluent & Comprehends
       type_label = 3  # Non-Fluent
       flu_label = 0  # Non-Fluent
       com_label = 0  # Comprehends
       
   elif aphasia_type in ['global', 'isolation']:
       type = "Global" if aphasia_type == 'global' else "Isolation"
       status = 'Aphasia'
       ct_label = 1
       wab_label = 4  # Non-Fluent & Not Comprehends
       type_label = 3  # Non-Fluent
       flu_label = 0  # Non-Fluent
       com_label = 1  # Not Comprehends
   
   else:
       return None, None, None, None, None, None

   return type, status, ct_label, wab_label, type_label, flu_label, com_label

In [3]:
video_col = ['NOSE_x', 'NOSE_y', 'NOSE_z', 'LEFT_EYE_INNER_x','LEFT_EYE_INNER_y', 'LEFT_EYE_INNER_z', 'LEFT_EYE_x', 'LEFT_EYE_y','LEFT_EYE_z', 
             'LEFT_EYE_OUTER_x', 'LEFT_EYE_OUTER_y','LEFT_EYE_OUTER_z', 'RIGHT_EYE_INNER_x', 'RIGHT_EYE_INNER_y','RIGHT_EYE_INNER_z', 
             'RIGHT_EYE_x', 'RIGHT_EYE_y', 'RIGHT_EYE_z','RIGHT_EYE_OUTER_x', 'RIGHT_EYE_OUTER_y', 'RIGHT_EYE_OUTER_z','LEFT_EAR_x', 
             'LEFT_EAR_y', 'LEFT_EAR_z', 'RIGHT_EAR_x', 'RIGHT_EAR_y','RIGHT_EAR_z', 'MOUTH_LEFT_x', 'MOUTH_LEFT_y', 'MOUTH_LEFT_z',
             'MOUTH_RIGHT_x', 'MOUTH_RIGHT_y', 'MOUTH_RIGHT_z', 'LEFT_SHOULDER_x','LEFT_SHOULDER_y', 'LEFT_SHOULDER_z', 'RIGHT_SHOULDER_x',
             'RIGHT_SHOULDER_y', 'RIGHT_SHOULDER_z', 'LEFT_ELBOW_x', 'LEFT_ELBOW_y','LEFT_ELBOW_z', 'RIGHT_ELBOW_x', 'RIGHT_ELBOW_y', 
             'RIGHT_ELBOW_z','LEFT_WRIST_x', 'LEFT_WRIST_y', 'LEFT_WRIST_z', 'RIGHT_WRIST_x','RIGHT_WRIST_y', 'RIGHT_WRIST_z', 'LEFT_PINKY_x', 
             'LEFT_PINKY_y','LEFT_PINKY_z', 'RIGHT_PINKY_x', 'RIGHT_PINKY_y', 'RIGHT_PINKY_z','LEFT_INDEX_x', 'LEFT_INDEX_y', 'LEFT_INDEX_z',
             'RIGHT_INDEX_x','RIGHT_INDEX_y', 'RIGHT_INDEX_z', 'LEFT_THUMB_x', 'LEFT_THUMB_y','LEFT_THUMB_z', 'RIGHT_THUMB_x', 'RIGHT_THUMB_y','RIGHT_THUMB_z']

audio_col = ['F0semitoneFrom27.5Hz_sma3nz_amean','F1amplitudeLogRelF0_sma3nz_amean','F1bandwidth_sma3nz_amean','F1frequency_sma3nz_amean',
             'F2amplitudeLogRelF0_sma3nz_amean','F2bandwidth_sma3nz_amean','F2frequency_sma3nz_amean','F3amplitudeLogRelF0_sma3nz_amean',
             'F3bandwidth_sma3nz_amean','F3frequency_sma3nz_amean','HNRdBACF_sma3nz_amean','alphaRatioV_sma3nz_amean',
             'hammarbergIndexV_sma3nz_amean','jitterLocal_sma3nz_amean','logRelF0-H1-A3_sma3nz_amean','logRelF0-H1-H2_sma3nz_amean',
             'loudness_sma3_amean','mfcc1_sma3_amean','mfcc2_sma3_amean','mfcc3_sma3_amean','mfcc4_sma3_amean','shimmerLocaldB_sma3nz_amean',
             'slopeV0-500_sma3nz_amean','slopeV500-1500_sma3nz_amean','spectralFlux_sma3_amean']

with open('D:/aphasia/MMATD/data_preprocessing/metadata.json','r') as f:
    previous_metadata = json.load(f)
    metadata = {}
    for k,v in previous_metadata.items():
        metadata[k.split('/')[1]] = v

dataset_chunk = {"user_name": {}, 'status': {}, 'chunk_id': {}, 
                 "type": {}, "sex": {}, "asr_body_pre": {}, 
                 "ct_label": {}, "wab_label": {}, "type_label": {},
                 "flu_label": {}, "com_label": {}, "data_id": {},
                 "duration": {}}

dirs = glob('D:/aphasia/dataset/tokens/text_bind/*')
dfs = []
audio_feats =[]


data_id = 0
for dir in tqdm(dirs):
    # aling the features
    txt_img_path = os.path.join(dir,'txt_img_paths.json')
    opensmile_path = os.path.join(dir.replace('text_bind','opensmile_bind'),'txt_img_paths.json')
    pose_path = os.path.join(dir.replace('text_bind','pose_bind'),'txt_img_paths.json')

    row = {}
    row['txt_img_path'] = txt_img_path 
    row['opensmile_path'] = opensmile_path
    row['pose_path'] = pose_path

    df_txt = pd.read_json(row['txt_img_path'])
    df_aud = pd.read_json(row['opensmile_path'])[audio_col].interpolate()
    df_vid = pd.read_json(row['pose_path'])[video_col].interpolate()

    df_txt = pd.concat([df_txt,df_vid],axis=1)
    df_txt = pd.concat([df_txt,df_aud],axis=1)

    fn = df_txt.iloc[0]['token_img_path'].split('\\')[1]
    metadata_info = metadata[fn]
    aphasia_type, status, ct_label, wab_label, type_label, flu_label, com_label = get_labels(metadata_info['aphasia_type'])

    if aphasia_type is None:
        continue
    # extract information for dataset_chunk
    for i in range(0, len(df_txt), 50):
        with open(f'D:/aphasia/dataset/transcripts/{fn}.json', 'r') as f:
            transcript = json.load(f)
        try:
            if i + 50 <= len(df_txt):
                dfs.append(df_txt.iloc[i:i+50])
            else:
                continue
            data_id += 1
            dataset_chunk['duration'][data_id] = transcript['chunks'][i//50]['end'] - transcript['chunks'][i//50]['start']
            asr_body_pre = ' '.join(df_txt.iloc[i:i+50]['token'])

            dataset_chunk['data_id'][data_id] = data_id
            dataset_chunk['user_name'][data_id] = fn

            dataset_chunk['chunk_id'][data_id] = i % 50
            dataset_chunk['asr_body_pre'][data_id] = ' '.join(df_txt.iloc[i:i+50]['token']).replace('\u2014', ' ')
        

            dataset_chunk['sex'][data_id] = metadata_info['sex']
            dataset_chunk['data_id'][data_id] = data_id

            dataset_chunk['status'][data_id] = status
            dataset_chunk['type'][data_id] = aphasia_type
            dataset_chunk['ct_label'][data_id] = ct_label
            dataset_chunk['wab_label'][data_id] = wab_label
            dataset_chunk['type_label'][data_id] = type_label
            dataset_chunk['flu_label'][data_id] = flu_label
            dataset_chunk['com_label'][data_id] = com_label
        except:
            continue

    # TODO: df_txt들 리스트에서 하나씩 꺼낼 때 또 50개 단위로 청킹하고 각 청크마다 인접행렬 생성하기

100%|██████████| 1012/1012 [02:29<00:00,  6.78it/s]


In [17]:
with open('D:/aphasia/MMATD/dataset/dataset_chunk50.json','w') as f:
    json.dump(dataset_chunk,f, indent=4)

In [4]:
def add_special_tokens_to_features(features):
    _, feature_dim = features.shape
    
    cls_token = np.random.normal(0, 0.02, (1, feature_dim))
    eos_token = np.random.normal(0, 0.02, (1, feature_dim))
    
    augmented_features = np.concatenate([cls_token, features, eos_token], axis=0)
    
    return augmented_features

audio_feats_with_additional_tokens = []
video_feats_with_additional_tokens = []
audio_feats = []
video_feats = []
chunk_size = 50

for df in tqdm(dfs):
    audio_feat = np.array(df[audio_col].values)
    video_feat = np.array(df[video_col].values)
    audio_feats.append(audio_feat)
    video_feats.append(video_feat)

    audio_feats_with_additional_tokens.append(add_special_tokens_to_features(audio_feat))
    video_feats_with_additional_tokens.append(add_special_tokens_to_features(video_feat))

audio_feats = np.stack(audio_feats)
video_feats = np.stack(video_feats)
audio_feats_with_additional_tokens = np.stack(audio_feats_with_additional_tokens)
video_feats_with_additional_tokens = np.stack(video_feats_with_additional_tokens)

100%|██████████| 8066/8066 [00:10<00:00, 792.31it/s]


In [5]:
np.save('D:/aphasia/MMATD/dataset/opensmile_chunk52_feat.npy', audio_feats_with_additional_tokens)
np.save('D:/aphasia/MMATD/dataset/pose_chunk52_feat.npy', video_feats_with_additional_tokens)
np.save('D:/aphasia/MMATD/dataset/opensmile_chunk50_feat.npy', audio_feats)
np.save('D:/aphasia/MMATD/dataset/pose_chunk50_feat.npy', video_feats)

In [34]:
print('Aphasia Type')
print('Control: ', len([v for v in dataset_chunk['type'].values() if v == 'Control']))
print('Anomic: ', len([v for v in dataset_chunk['type'].values() if v == 'Anomic']))
print('Conduction: ', len([v for v in dataset_chunk['type'].values() if v == 'Conduction']))
print('Wernicke: ', len([v for v in dataset_chunk['type'].values() if v == 'Wernicke']))
print('Sensory: ', len([v for v in dataset_chunk['type'].values() if v == 'Trans. Sensory']))
print('Broca: ', len([v for v in dataset_chunk['type'].values() if v == 'Broca']))
print('Motor: ', len([v for v in dataset_chunk['type'].values() if v == 'Trans. Motor']))

print()

print('Subjects')
print('Control: ', len(np.unique([v for k, v in dataset_chunk['user_name'].items() if dataset_chunk['type'][k] == 'Control'])))
print('Anomic: ', len(np.unique([v for k, v in dataset_chunk['user_name'].items() if dataset_chunk['type'][k] == 'Anomic'])))
print('Conduction: ', len(np.unique([v for k, v in dataset_chunk['user_name'].items() if dataset_chunk['type'][k] == 'Conduction'])))
print('Wernicke: ', len(np.unique([v for k, v in dataset_chunk['user_name'].items() if dataset_chunk['type'][k] == 'Wernicke'])))
print('Sensory: ', len(np.unique([v for k, v in dataset_chunk['user_name'].items() if dataset_chunk['type'][k] == 'Trans. Sensory'])))
print('Broca: ', len(np.unique([v for k, v in dataset_chunk['user_name'].items() if dataset_chunk['type'][k] == 'Broca'])))
print('Motor: ', len(np.unique([v for k, v in dataset_chunk['user_name'].items() if dataset_chunk['type'][k] == 'Trans. Motor'])))  

print()

print('Duration')
print('Control: ', np.mean([v for k, v in dataset_chunk['duration'].items() if dataset_chunk['type'][k] == 'Control']))
print('Anomic: ', np.mean([v for k, v in dataset_chunk['duration'].items() if dataset_chunk['type'][k] == 'Anomic']))
print('Conduction: ', np.mean([v for k, v in dataset_chunk['duration'].items() if dataset_chunk['type'][k] == 'Conduction']))
print('Wernicke: ', np.mean([v for k, v in dataset_chunk['duration'].items() if dataset_chunk['type'][k] == 'Wernicke']))
print('Sensory: ', np.mean([v for k, v in dataset_chunk['duration'].items() if dataset_chunk['type'][k] == 'Trans. Sensory']))  
print('Broca: ', np.mean([v for k, v in dataset_chunk['duration'].items() if dataset_chunk['type'][k] == 'Broca']))
print('Motor: ', np.mean([v for k, v in dataset_chunk['duration'].items() if dataset_chunk['type'][k] == 'Trans. Motor']))  
print('Total: ', np.mean([v for k, v in dataset_chunk['duration'].items()]))                                                                                                 
                                                                                                        

Aphasia Type
Control:  3364
Anomic:  1871
Conduction:  1171
Wernicke:  439
Sensory:  3
Broca:  1110
Motor:  42

Subjects
Control:  318
Anomic:  247
Conduction:  127
Wernicke:  56
Sensory:  1
Broca:  186
Motor:  11

Duration
Control:  19.850912604042808
Anomic:  34.72995724211651
Conduction:  27.382083689154566
Wernicke:  28.12956719817768
Sensory:  19.253333333333334
Broca:  38.360369369369366
Motor:  45.804761904761904
Total:  27.59163625


In [35]:
np.unique(list(dataset_chunk['type'].values()))

array(['Anomic', 'Broca', 'Conduction', 'Control', 'Trans. Motor',
       'Trans. Sensory', 'Wernicke'], dtype='<U14')

In [36]:
import pandas as pd
from collections import defaultdict
import numpy as np


def calculate_co_occurrence_matrix(dfs, disfluency_tokens, video_col, audio_col, std_multiplier=1.0, min_value=0.01):
    """
    비디오와 오디오 특성을 따로 정규화하여 co-occurrence 계산
    Args:
        dfs: 데이터프레임 리스트
        disfluency_tokens: 분석할 토큰 리스트
        video_col: 비디오 특성 컬럼 리스트
        audio_col: 오디오 특성 컬럼 리스트
        std_multiplier: 표준편차의 몇 배를 threshold로 사용할지 결정하는 값
        min_value: 모든 co-occurrence 값에 더할 최솟값
    """
    print("Concatenating all dataframes...")
    combined_df = pd.concat(dfs, axis=0).reset_index(drop=True)
    
    # 토큰 마스크 생성
    token_masks = {}
    for token in tqdm(disfluency_tokens, desc="Creating token masks"):
        token_masks[token] = (combined_df['token'].str.lower() == token)
        
    # 각 토큰의 총 등장 횟수 계산
    token_counts = {
        token: mask.sum() 
        for token, mask in token_masks.items()
    }
    
    print("Calculating co-occurrence matrix...")
    co_occurrences = {
        'video': defaultdict(dict),
        'audio': defaultdict(dict)
    }
    
    with tqdm(total=len(video_col) + len(audio_col), desc="Processing features") as pbar:
        # 비디오 특성 처리
        video_occurrences = 0
        for col in video_col:
            if col in combined_df.columns:
                # 변화량 계산
                changes = np.abs(combined_df[col].diff())
                
                # 변화량의 표준편차 계산
                changes_std = changes.std()
                if changes_std > 0:
                    # 표준편차의 일정 배수를 threshold로 사용
                    threshold = changes_std * std_multiplier
                    changes_mask = (changes > threshold)
                    
                    video_occurrences += np.sum(changes_mask)
                    
                    for token in disfluency_tokens:
                        co_occurrences['video'][col][token] = np.sum(
                            token_masks[token] & changes_mask
                        )
                else:
                    # 변화가 없는 경우
                    print(f"\nWarning: No variation in feature '{col}'")
                    for token in disfluency_tokens:
                        co_occurrences['video'][col][token] = 0
                        
                pbar.update(1)
        
        # 오디오 특성 처리 (기존과 동일)
        audio_occurrences = 0
        for col in audio_col:
            if col in combined_df.columns:
                mean = combined_df[col].mean()
                std = combined_df[col].std()
                if std > 0:
                    z_scores = np.abs((combined_df[col] - mean) / std)
                    z_score_mask = (z_scores > 1.0)
                    audio_occurrences += np.sum(z_score_mask)
                    
                    for token in disfluency_tokens:
                        co_occurrences['audio'][col][token] = np.sum(
                            token_masks[token] & z_score_mask
                        )
                pbar.update(1)
    
    print(f"\nTotal video changes detected: {video_occurrences}")
    
    # 정규화 부분 (기존과 동일)
    with tqdm(total=len(video_col) + len(audio_col), desc="Normalizing features") as pbar:
        normalized_co_occurrences = {}
        
        # 비디오 특성 정규화 및 최솟값 더하기
        for col in video_col:
            normalized_co_occurrences[col] = {}
            for token in disfluency_tokens:
                if video_occurrences > 0:
                    value = co_occurrences['video'][col][token] / video_occurrences
                    normalized_co_occurrences[col][token] = value + min_value
                else:
                    normalized_co_occurrences[col][token] = min_value
            pbar.update(1)
        
        # 오디오 특성 정규화 및 최솟값 더하기
        for col in audio_col:
            normalized_co_occurrences[col] = {}
            for token in disfluency_tokens:
                if audio_occurrences > 0:
                    value = co_occurrences['audio'][col][token] / audio_occurrences
                    normalized_co_occurrences[col][token] = value + min_value
                else:
                    normalized_co_occurrences[col][token] = min_value
            pbar.update(1)
    
    # 통계 출력 (기존과 동일)
    all_values = [
        normalized_co_occurrences[col][token]
        for col in normalized_co_occurrences
        for token in disfluency_tokens
    ]
    
    print("\nCo-occurrence statistics:")
    print(f"Total tokens processed: {sum(token_counts.values())}")
    print(f"Total video changes: {video_occurrences}")
    print(f"Total audio significant changes: {audio_occurrences}")
    print(f"\nValue distribution after adding min_value ({min_value}):")
    print(f"Mean: {np.mean(all_values):.4f}")
    print(f"Min: {np.min(all_values):.4f}")
    print(f"Max: {np.max(all_values):.4f}")
    
    print("\nToken statistics:")
    for token in disfluency_tokens:
        if token_counts[token] > 0:
            print(f"{token}: {token_counts[token]} occurrences")
            video_avg = np.mean([normalized_co_occurrences[col][token] for col in video_col])
            audio_avg = np.mean([normalized_co_occurrences[col][token] for col in audio_col])
            print(f"  - Avg video co-occurrence: {video_avg:.4f}")
            print(f"  - Avg audio co-occurrence: {audio_avg:.4f}")
    
    return normalized_co_occurrences


disfluency_tokens = json.load(open('D:/aphasia/MMATD/dataset/_disfluency_tk_300.json', 'r'))

co_occurrences = calculate_co_occurrence_matrix(
    dfs, 
    disfluency_tokens, 
    video_col, 
    audio_col,
    std_multiplier=1.5  # 표준편차의 1배를 threshold로 사용
)

Concatenating all dataframes...


Creating token masks: 100%|██████████| 300/300 [00:41<00:00,  7.19it/s]


Calculating co-occurrence matrix...


Processing features: 100%|██████████| 94/94 [00:33<00:00,  2.82it/s]



Total video changes detected: 1866673


Normalizing features: 100%|██████████| 94/94 [00:00<00:00, 3470.06it/s]


Co-occurrence statistics:
Total tokens processed: 294098
Total video changes: 1866673
Total audio significant changes: 2614170

Value distribution after adding min_value (0.01):
Mean: 0.0101
Min: 0.0100
Max: 0.0195

Token statistics:
[*]: 63195 occurrences
  - Avg video co-occurrence: 0.0130
  - Avg audio co-occurrence: 0.0159
the: 21527 occurrences
  - Avg video co-occurrence: 0.0107
  - Avg audio co-occurrence: 0.0124
and: 22026 occurrences
  - Avg video co-occurrence: 0.0111
  - Avg audio co-occurrence: 0.0122
to: 10846 occurrences
  - Avg video co-occurrence: 0.0102
  - Avg audio co-occurrence: 0.0112
um: 462 occurrences
  - Avg video co-occurrence: 0.0100
  - Avg audio co-occurrence: 0.0101
she: 10053 occurrences
  - Avg video co-occurrence: 0.0104
  - Avg audio co-occurrence: 0.0113
a: 7431 occurrences
  - Avg video co-occurrence: 0.0102
  - Avg audio co-occurrence: 0.0108
her: 6111 occurrences
  - Avg video co-occurrence: 0.0102
  - Avg audio co-occurrence: 0.0106
was: 4702 occ

In [37]:
def create_feature_adjacency_matrices(dfs, disfluency_tokens, video_col, audio_col, co_occurrences):
    n_samples = len(dfs)
    n_columns = len(video_col) + len(audio_col)
    n_disfluency = len(disfluency_tokens)
    
    final_matrix = np.zeros((n_samples, n_columns, n_disfluency), dtype=np.float16)
    
    print("\nCreating adjacency matrices...")
    for df_idx, df in enumerate(tqdm(dfs)):
        token_indices = np.array([
            disfluency_tokens.index(token.lower()) if token.lower() in disfluency_tokens else -1 
            for token in df['token']
        ])
        valid_token_mask = token_indices != -1
        
        # 비디오 특성 처리
        changes = np.zeros((len(df), len(video_col)))
        for j, col in enumerate(video_col):
            if col in df.columns:
                diff = np.abs(df[col].diff().fillna(0))
                changes[:, j] = (diff > 0).astype(np.float32)
        
        for j, col in enumerate(video_col):
            if col in df.columns:
                change_mask = changes[:, j] > 0
                valid_positions = valid_token_mask & change_mask
                if np.any(valid_positions):
                    token_idx = token_indices[valid_positions]
                    for pos, tok_idx in zip(np.where(valid_positions)[0], token_idx):
                        final_matrix[df_idx, j, tok_idx] = co_occurrences[col][disfluency_tokens[tok_idx]]
        
        # 오디오 특성 처리
        for j, col in enumerate(audio_col, start=len(video_col)):
            if col in df.columns:
                values = df[col].values
                if len(values[~np.isnan(values)]) > 0:  # NaN이 아닌 값이 있는 경우에만 처리
                    mean = np.nanmean(values)
                    std = np.nanstd(values)
                    
                    if std > 0:  # 표준편차가 0보다 큰 경우에만 z-score 계산
                        z_scores = np.abs((values - mean) / std)
                        significant_mask = (z_scores > 1.0) & ~np.isnan(z_scores)
                        
                        valid_positions = valid_token_mask & significant_mask
                        if np.any(valid_positions):
                            token_idx = token_indices[valid_positions]
                            for pos, tok_idx in zip(np.where(valid_positions)[0], token_idx):
                                final_matrix[df_idx, j, tok_idx] = co_occurrences[col][disfluency_tokens[tok_idx]]
    
    return final_matrix

adj_matrix = create_feature_adjacency_matrices(dfs, disfluency_tokens, video_col, audio_col, co_occurrences)
print(f"Adjacency matrix shape: {adj_matrix.shape}")



Creating adjacency matrices...


  0%|          | 0/8066 [00:00<?, ?it/s]

100%|██████████| 8066/8066 [08:28<00:00, 15.86it/s]

Adjacency matrix shape: (8066, 94, 300)


In [38]:
np.save('D:/aphasia/MMATD/dataset/adj_chunk50_300_duration.npy', adj_matrix)